<a href="https://colab.research.google.com/github/kbw612/Fantasy/blob/main/colab/OffensiveLineAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import isin
import pandas as pd

incorrect_team_abbrev_to_correct_team_abbrev_map = {
  "KCC": "KC",
  "SFO": "SF",
  "TBB": "TB",
  "NOS": "NO",
  "NEP": "NE",
  "LVR": "LV",
  "JAC": "JAX",
  "GBP": "GB",
  "ARZ": "ARI"
}

def clean_team_abbrev(team_abbrev):
  correct_team_abbrev = incorrect_team_abbrev_to_correct_team_abbrev_map.get(team_abbrev, "")

  if correct_team_abbrev == "":
        return team_abbrev

  return correct_team_abbrev

pd.set_option('display.max_colwidth', None)  # Disable column width limit
pd.set_option('display.expand_frame_repr', False)
red_color = "\033[91m"
default_color = "\033[0m"
bold_text = "\033[1m"
default_text = "\033[0m"
offensive_line_positions = ["Left Tackle","Left Guard","Center","Right Guard","Right Tackle"]

def get_injury_report(week):
  injury_report_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/nfl-injury-report-week{week}.csv")

  for row in injury_report_df.index:
    injury_report_df["Est. Return"] = pd.to_datetime(injury_report_df.at[row, "Est. Return"])

  return injury_report_df

def print_team_offensive_line(ol_df, injury_report_df, weeks):
  print("Projected Starting Offensive Line")

  teams = sorted(ol_df['Team'].unique())
  for team in teams:
    team_condition = ol_df["Team"] == team
    weeks_condition = ol_df["Week"].isin(weeks)

    ol_team_df = ol_df[team_condition & weeks_condition]
    different_rows = ol_team_df.drop_duplicates(subset=offensive_line_positions, keep=False)

    print("")

    if len(different_rows) > 0:
      print(f"{red_color}{team} ******* different week {current_week} starting OL *******{default_color}")
      print(ol_team_df.to_string(index=False))
    else:
      print(f"{bold_text}{team}{default_text}")
      print(ol_team_df[ol_team_df.duplicated(subset=offensive_line_positions, keep="first")].to_string(index=False))

    team_condition = injury_report_df["Team"] == team
    team_injury_report_df = injury_report_df[team_condition]

    if len(team_injury_report_df) > 0:
      positions = ["OT", "C", "OG"]
      estimated_return_condition = team_injury_report_df["Est. Return"] < pd.to_datetime('2024-02-12')
      positions_condition = team_injury_report_df["Pos"].isin(positions)

      team_injury_report_df = team_injury_report_df[positions_condition & estimated_return_condition]

      if len(team_injury_report_df) > 0:
        sorted_df = team_injury_report_df.sort_values(by=["Pos"], ascending=[True])

        print("")
        print(f"OL Injuries")
        print(sorted_df.to_string(index=False))

def print_rankings(week, ol_football_guys_df):
  # clean team abbreviation

  #Week,O-Line Ranking,Team,Opponent,aFPA,Difference
  ol_4For4_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/offensive-line-4For4.csv?1=2")

  for row in ol_4For4_df.index:
    ol_4For4_df.at[row, "Team"] = clean_team_abbrev(ol_4For4_df.at[row, "Team"])

  ol_4For4_df = ol_4For4_df[ol_4For4_df["Week"] == week]
  ol_4For4_df = ol_4For4_df.drop(columns=["Opponent", "aFPA", "Difference", "Week"], axis=1).rename(columns={"O-Line Ranking": "Rank4For4", "Team": "Team4For4"})

  #Week,Rank,Team_ID,Name,Time To Throw,Time To Throw Rank,Hurry Rate,Hurry Rate Rank,Run Blocking Rating,Run Blocking Rating Rank,Sacks Allowed
  ol_player_profiler_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/offensive-line-PlayerProfiler.csv")

  for row in ol_player_profiler_df.index:
    ol_player_profiler_df.at[row, "Team_ID"] = clean_team_abbrev(ol_player_profiler_df.at[row, "Team_ID"])

  ol_player_profiler_df = ol_player_profiler_df[ol_player_profiler_df["Week"] == week]
  ol_player_profiler_df = ol_player_profiler_df.drop(columns=["Name", "Time To Throw", "Time To Throw Rank", "Hurry Rate", "Hurry Rate Rank", "Run Blocking Rating", "Run Blocking Rating Rank", "Sacks Allowed", "Week"], axis=1).rename(columns={"Rank": "RankPlayerProfiler", "Team_ID": "TeamPlayerProfiler"})

  for row in ol_football_guys_df.index:
    ol_football_guys_df.at[row, "Team"] = clean_team_abbrev(ol_football_guys_df.at[row, "Team"])

  ol_football_guys_df = ol_football_guys_df[ol_football_guys_df["Week"] == week]
  #Week,Rank,Team,Left Tackle,Left Guard,Center,Right Guard,Right Tackle
  ol_football_guys_df = ol_football_guys_df.drop(columns=["Left Tackle", "Left Guard", "Center", "Right Guard", "Right Tackle", "Week"], axis=1).rename(columns={"Rank": "RankFootballGuys", "Team": "TeamFootballGuys"})

  rank_df = pd.merge(ol_4For4_df, ol_player_profiler_df, left_on="Team4For4", right_on="TeamPlayerProfiler", how="left")
  rank_df = rank_df.drop(columns=["TeamPlayerProfiler"], axis=1)
  rank_df = pd.merge(rank_df, ol_football_guys_df, left_on="Team4For4", right_on="TeamFootballGuys", how="left")
  rank_df = rank_df.drop(columns=["TeamFootballGuys"], axis=1)
  rank_df = rank_df.rename(columns={"Team4For4": "Team"})

  average_rank = rank_df[["Rank4For4", "RankPlayerProfiler", "RankFootballGuys"]].mean(axis=1)
  rank_df["AvgRank"] = round(average_rank, 2)
  rank_df = rank_df[["Team", "AvgRank", "Rank4For4", "RankPlayerProfiler", "RankFootballGuys"]]
  rank_df = rank_df.sort_values(by=["AvgRank"], ascending=[True])

  print(rank_df.to_string(index=False))

current_week = 16
weeks = [current_week - 1, current_week]
ol_football_guys_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/offensive-line-FootballGuys.csv?1=2")
injury_report_df = get_injury_report(current_week)

print_rankings(5, ol_football_guys_df)
print_team_offensive_line(ol_football_guys_df, injury_report_df, weeks)


Team  AvgRank  Rank4For4  RankPlayerProfiler  RankFootballGuys
 PHI     1.00          1                   1                 1
 DET     2.33          3                   2                 2
 DAL     5.33          4                   8                 4
 IND     7.33         10                   4                 8
 ATL     8.33         12                   7                 6
  KC     8.67          2                  21                 3
  TB     9.00          7                  11                 9
 BUF     9.33         15                   6                 7
 MIN    10.67         17                   3                12
 DEN    11.00         14                   5                14
 CLE    12.00          6                  19                11
 LAC    12.67          5                  16                17
  SF    13.00         11                  23                 5
 BAL    14.67         16                  13                15
 MIA    15.33          9                  12           